# The IPDL model

The IPDL model is a generalization of the nested logit model where each alternative may belong to more than one nest.
In this model, decision makers have to choose between $J$
different discrete options, $\{1,2,...,J\}$. An option might be a car.
For each option, we observe a vector of characteristics,
$x_{j}\in\mathbb{R}^{K}$, and for each individual $i=1,...,N$, we
observe the chosen alternative, $a_{i}\in\{1,...,J\}$. If individuals
face different alternatives, e.g. if the prices or characteristics of
cars available were different, then characteristics also vary across
individuals, $x_{ij}\in\mathbb{R}^{K}$.

Our model for utility takes the form $u_{ij}=x_{ij}\beta_0$, where $\beta_0$ is an unknown vector of parameters. We can write the vector of utilities $u_i$ as $\mathbf{X}_i\beta_0$ as we did in the logit model. 

It will be convenient to define $y_i$ as the "one-hot" encoding of the choice $a_i$. Let $e_j$ denote the $j$'th canonical basis vector in $\mathbb R^J$, and set $y_i=e_{a_i}$.

## The perturbation function $\Omega$

In the following, a vector $z\in \mathbb R^d$ is always a column vector. We now construct the perturbation function. Let $\Delta=\left\{q\in \mathbb{R}^J_+: \sum_{j=1}^J q_j=1\right\}$ denote the probability simplex. For each group of nests $g=1,\ldots, G$, nest membership is denoted by the matrix $\Psi^g\in \mathbb R^{C_g\times J}$: $\Psi^g_{cj}=1$ if product $j$ belongs to nest $c$ and zero otherwise, and each product can only belong to one nest within each group, meaning that $\sum_{c=1}^{C_g}\Psi^g_{cj}=1$ for all $j$ and all $g$. The matrix-vector product $\Psi^gq$ is then
$$
\Psi^g q=\sum_j \Psi^{g}_{cj}q_j=\left(\begin{array}{c}
\sum_{j:\Psi^g_{1j}=1} q_j \\
\vdots \\
\sum_{j: \Psi^g_{C_gj}=1}q_j
\end{array}\right),
$$
and the vector $\Psi^gq$ is a vector of nest-specific choice probabilities, i.e. the sum of the probabilities within each nest. The IPDL perturbation function then has the form (where for a vector $z$, the logarithm is applied elementwise and $z'$ denote the transpose)
$$
\Omega(q|\lambda)= (1-\sum_{g=1}^G \lambda_g) q'\ln q +\sum_{g=1}^{G} \lambda_g \left(\Psi^g q \right)'\ln \left(\Psi^g q\right).
$$
Note that since $\Psi^g q$ denotes a probability distribution over the nests, the term $(\Psi^gq)'\ln (\Psi^gq)$ is the (negative) entropy of the probability distribution $\Psi^g q$. Note also that as each nest has at least one member, and $q$ is strictly positive, $\Psi^gq$ is also strictly positive. When the parameters $\lambda_g$ satisfy $\lambda_g>0$ and
$$
\sum_g \lambda_g<1,
$$
the function $\Omega(\cdot|\lambda)$ is a strictly convex function of $q$, and the utility maximization problem has a unique interior (meaning strictly positive choice probabilities) solution. When there is only one group of nests, $G=1$, then $\Omega$ induces the nested logit choice probabilities (note though that the nested logit model is often parameterized in terms of the nesting parameter $\mu=1-\lambda$ instead!). 

It will be convenient to define a choice probability function for a given vector of payoffs $u$ as
$$
P(u|\lambda)=\arg \max_{q\in \Delta}q'u-\Omega(q|\lambda).
$$
Letting $\theta$ denote the full vector of parameters, $\theta=(\beta',\lambda')'$, the individual choice probabilities is a function of the matrix $\mathbf{X}_i$ and the parameters $\theta$, as
$$
p(\mathbf{X}_i,\theta)=\arg\max_{q\in \Delta}q'\mathbf{X}_i \beta-(1-\sum_{g=1}^G\lambda_g)q'\ln q-\sum_{g=1}^G\lambda_g \left(\Psi^g q \right)'\ln \left(\Psi^g q\right)
$$

## Model solution and maximum likelihood

While it is possible to solve for the choice probabilities explicitly by maximizing utility, Fosgerau and Nielsen (2021) suggest a contraction mapping approach which is conceptually simpler. Suppose we are evaluating the likelihood at some guess of the parameters $\theta=(\beta',\lambda')$. Let $u_i=\mathbf{X}_i\beta$, and let $q_i^0$ denote some initial vector of choice probabilities e.g. $q^0_i=\frac{e^{u_i}}{\sum_{j'=1}^Je^{u_{ij'}}}$, we update the choice probabilities according to the formula
$$
v_i^{k} =u_i+\ln q_i^{k-1}-\Gamma'\ln (\Gamma q_i^{k-1})\\
q_i^{k} = \frac{e^{v_i^{k}}}{\sum_{j=1}^J e^{v_{ij}^{k}}},
$$
they show that $\lim_{k\rightarrow \infty}q_i^k=p(\mathbf{X}_i,\theta)$ for any starting value $q^0_i$ in the interior of $\Delta$. For numerical stability, it can be a good idea to also do max-rescaling of $v^k_i$ at every iteration.

Let $p$ denote the solution to the utility maximization problem. Formally, the Kullback-Leibler divergence $D_{KL}(p||q)=p'\ln \frac{p}{q}$ decays linearly with each iteration,
$$
D_{KL}(p||q^{k+1})\leq (1-\sum_g \lambda_g)D_{KL}(p||q^k),
$$
Noting that $(1-\sum_g \lambda_g)\in [0,1)$ by assumption.

The log-likelihood contribution is
$$
\ell_i(\theta)=y_i'\ln p(\mathbf{X}_i,\theta),
$$
and an estimation routine must therefore have a function that - given $\mathbf{X}_i$ and $\theta$ - calculates $u_i=\mathbf{X}_i\beta$ and constructs $\Gamma$, and then calls the fixed point routine described above. That routine will return $p(\mathbf{X}_i,\theta)$, and we can then evaluate $\ell_i(\theta)$.

## loglikelihood estimation

The log-likelihood contribution is
$$
\ell_i(\theta)=y_i'\ln p(\mathbf{X}_i,\theta).
$$

For maximizing the likelihood, we want the derivates at some $\theta=(\beta',\lambda')$. Let $q_i=p(\mathbf{X}_i,\theta)$, then we have
$$
\nabla_\theta \ln p(\mathbf{X}_i,\theta)=\mathrm{diag}(q_i)^{-1}\left(\nabla_{qq}^2\Omega(q_i|\lambda)^{-1}-q_iq_i' \right)\left[\mathbf{X}_i,-\nabla_{q,\lambda}^2 \Omega(q_i|\lambda)\right]
$$
Note that the first two components is the elasticity $\nabla_u \ln P(u|\lambda)$ and the last term is a block matrix of size $J\times dim(\theta)$. The derivative of the log-likelihood function can be obtained from this as
$$
\nabla_\theta \ell_i(\theta)=\nabla_\theta \ln p(\mathbf{X}_i,\theta)' y_i \\
$$


It may also be useful to use the BHHH algorithm for maximizing the likelihood, which is given by
$$
        \theta^{k+1}=\theta^k-\alpha_k \left(\frac{1}{n}\sum_i \nabla_\theta \ell_i(\theta^k)\nabla_\theta \ell_i(\theta^k)'\right)^{-1} \left( \frac{1}{n}\sum_i \nabla_\theta \ell_i(\theta^k)\right)
$$

where $\alpha_k$ is a step-size found by e.g. backtracking line search.

# Max-rescaling for numerical stability

Let $\alpha$ be a scalar, and let $\iota$ be the all-ones vector in $\mathbb R^J$. Note that $q'(u+\alpha\iota)=q'u+(q'\iota)\alpha=q'u+\alpha$, since $q$ sums to one. For this reason, $\alpha$ does not enter into the utility maximization when calculating $P(u+\alpha\iota|\lambda)$, and we have $P(u+\alpha\iota|\lambda)=P(u|\lambda)$.

This allows us to re-scale the utilities just as in the logit model, since $P(u-(\max_{j}u_j)\iota|\lambda)=P(u|\lambda)$. The numerical benefits of this approach carry over to the IPDL model.

## Gradient and Hessian

For purposes of computing the gradient and Hessian of $\Omega$, it is convenient to define
$$
\Gamma=\left(\begin{array}{c}
(1-\sum_g \lambda_g)I_J\\
\lambda_1 \Psi^1\\
\vdots\\
\lambda_G \Psi^G
\end{array}\right)
$$
where $I_J$ is the identity matrix in $\mathbb R^J$. The matrix $\Gamma$ is a block matrix with $J+\sum_g C_g$ rows and $J$ columns. Note that 

$$
\Gamma q=\left(\begin{array}{c}
(1-\sum_g\lambda_g)q \\
\lambda_1\Psi^g q\\
\vdots \\
\lambda_G \Psi^Gq
\end{array}\right)>0
$$
if $q>0$.
# Exercise! Show that
$$
\Omega(q|\lambda)=(\Gamma q)'\ln (\Gamma q)+c\\
\nabla_q \Omega(q|\lambda)=\Gamma'\ln (\Gamma q)+\iota\\
\nabla^2_{qq}\Omega(q|\lambda)=\Gamma'\mathrm{diag}(\Gamma q)^{-1}\Gamma,
$$
where $c$ is a scalar that depends on $\lambda$ but on $q$ and therefore does not affect the utility maximization problem, $\iota=(1,\ldots,1)'\in \mathbb R^J$ is the all-ones vector and $\mathrm{diag}(z)$ is a diagonal matrix with the elements of $z$ on the diagonal. For extra points, calculate the gradient and Hessian directly using vector chain rules instead of elementwise.

## Demand derivatives and price Elasticity

While the demand derivatives in the IPDL model are not quite as simple as in the logit model, they are still easy to compute. 
Let $q=P(u|\lambda)$, then
$$
\nabla_u P(u|\lambda)=\left(\nabla^2_{qq}\Omega(q|\lambda)\right)^{-1}-qq'
$$
where the $()^{-1}$ denotes the matrix inverse. The derivation is a not too difficult, but will not be covered here. 

Exercise: The constraint that the choice probabilities sum to one must imply that the derivatives sum to zero: $\nabla_u P(u|\lambda)\iota=\iota$. Show that this holds by using what we know about $\nabla^2_{qq}\Omega(q|\lambda)$.

The derivatives with respect to any $x_{ij\ell}$ can now easily be computed by the chain rule,
$$
    \frac{\partial P_j(u_i|\lambda)}{\partial x_{ik\ell}}=\frac{\partial P_j(u_i|\lambda)}{\partial u_{ik}}\frac{\partial u_{ik}}{\partial x_{ik\ell}}=\frac{\partial P_j(u_i|\lambda)}{\partial u_{ik}}\beta_\ell,
$$

Finally, moving to price elasticity is the same as in the logit model, if $x_{ik\ell}$ is the log price of product $k$ for individual $i$, then
$$
    \mathcal{E}_{jk}= \frac{\partial P_j(u_i|\lambda)}{\partial x_{ijk}}\frac{1}{P_j(u_i|\lambda)},
$$
we can also write this compactly as
$$
\nabla_u \ln P(u|\lambda)=\mathrm{diag}(P(u|\lambda))^{-1}\nabla_u P(u|\lambda).
$$

## Compensating variation in Perturbed Utility Models

The analogy to the Emax function in the perturbed utility model is the maximum attainable utility/surplus function
$$
W(u|\lambda)=\max_{q\in \Delta}q'u-\Omega(q|\lambda).
$$
As done in the logit exercise, we can re-scale this utility by the price coefficient, say $\beta_1$, in order to compute the compensating variation
$$
CV= \frac{1}{\beta_1}W(\tilde u|\lambda)-\frac{1}{\beta_1}W(u|\lambda)
$$

## Fosgerau, Kristensen \& Nielsen estimation algorithm


The maximum likelihood estimation approach for the IPDL model is computationally heavy. Here, we describe an alternative estimation algorithm which will end up recovering the maximum likelihood estimator at the end. Let $\hat W_i$ be a matrix of weights. The weighted least-squares estimator
$$
\hat \theta_{ls}=\arg \min_{\theta}\frac{1}{n}\sum_i\left( y_i-p(\mathbf{X}_i,\theta)\right)'\hat{W}_i\left(y_i-p(\mathbf{X}_i\theta)\right),
$$
is an alternative to the maximum likelihood estimator. It is well-known that if $\hat W_i$ converges to $\mathrm{diag}(p(\mathbf{X}_i,\theta_0))^{-1}$, then $\hat \theta_{ls}$ is fully efficient (has the same asymptotic variance as the maximum likelihood estimator). Computing $\hat \theta_{ls}$ is not really any simpler than computing $\hat \theta_{mle}$, however.

Let $\hat p_i$ be some initial estimator of the choice probabilities for individual $i$ e.g. those obtained from estimating the logit model earlier. A straightforward choice for the weight matrix is then $\hat{W}_i=\mathrm{diag}(\hat p_i)^{-1}$. We propose to minimize the least-squares criterion using an approximation to $p(\mathbf{X}_i,\theta)$
$$
\hat \theta_{FKN}=  \arg \min_{\theta}\frac{1}{n} \sum_i \left(y_i-\tilde p(\mathbf{X}_i,\theta)\right)'\hat W_i\left( y_i-\tilde p(\mathbf{X}_i,\theta)\right).
$$
The approximation $\tilde p$ has the form
$$
\tilde p(\mathbf{X}_i,\theta)=\hat p_i -\hat D_i( u_i - \nabla_q \Omega(\hat p_i|\lambda)),
$$
where $\hat D_i=\mathrm{diag}(\hat p_i)-\hat p_i (\hat p_i)'$. Why this works as an approximation is outside the scope of this note. Let's calculate what $\tilde p(\mathbf{X}_i,\theta)$ looks like. Well, we know that $u_i=\mathbf{X}_i\beta$, and
$$
\nabla_q \Omega(\hat p_i|\lambda)=\Gamma'\ln \Gamma q =(1-\sum_g \lambda_g) \ln \hat p_i +\sum_g \lambda_g (\Psi^g)'\ln \left(\Psi^g \hat p_i\right)+\iota
$$
Note that $\hat D_i\iota =0$, so $\iota$ can be ignored. Now we construct some regressors. Let $\hat Z_{i}$ be a matrix of size $J\times G$, defined by
$$
\hat Z_{ijg}=\ln \hat p_{ij}-[(\Psi^g)'\ln (\Psi^g \hat p_{i})]_j,
$$
then the gradient of $\Omega$ can be written in the form
$$
\nabla_q \Omega(\hat p_i|\lambda)= \ln \hat p_i- \hat Z_i\lambda+\iota.
$$
plugging these terms into the approximation gives
$$
\tilde p(\mathbf{X}_i,\theta)=\hat p_i-\hat D_i\left( \mathbf{X}_i\beta+\hat{Z}_i\lambda-\ln \hat p_i\right),
$$
Finally, letting $\hat G_i=[\hat D_i\mathbf{X}_i,\hat D_i\hat Z_i]$ be a matrix of size $J\times (K+G)$, and letting $\hat y_i=y_i-\hat p_i+\hat D_i\ln \hat p_i$, we can now rewrite the estimator as
$$
\hat{\theta}_{FKN}=\arg \min_{\theta\in \Theta}\frac{1}{n}\sum_i\left(\hat{y}_i-\hat G_i\theta\right)'\hat W_i\left(\hat y_i-\hat G_i\theta\right),
$$
which is a linear weighted least squares estimator, with unique solution
$$
\hat \theta_{FKN}=\left(\frac{1}{n}\sum_i \hat G_i'\hat W_i\hat G_i\right)^{-1}\left(\frac{1}{n}\sum_i \hat G_i'\hat W_i \hat y_i\right),
$$
(note however that this solution might not respect the parameter bounds on $\lambda$). We have thus completely avoided solving for the choice probabilities $p(\mathbf{X}_i,\theta)$.

## Iterated estimator

Let the estimator obtained in the earlier section be $\hat \theta^0$, we now construct a sequence of estimators $ (\hat\theta^k)_{k=1}^\infty$. The estimator $\hat \theta^0$ is biased as shown in the paper, but the bias can be removed by iterations. 

The steps of the iteration are
$$
\hat p_i=p(\mathbf{X}_i,\hat \theta^k)\\
\hat W_i=\mathrm{diag}(\hat p_i)^{-1}\\
\hat D_i=(\nabla^2_{qq}\Omega(\hat p_i|\hat {\lambda}^k)^{-1}-\hat p_i \hat p_i'\\
\hat{Z}_{ijg}=\ln \hat p_{ij}-[(\Psi^g)'\ln (\Psi^g \hat p_{i})]_j \\
\hat{G}_i =\hat D_i[\mathbf{X}_i,\hat Z_i] \\
\hat y_i =y_i-\hat p_i +\hat D_i\ln \hat p_i \\
\hat{\theta}^{k+1}=\left(\frac{1}{n}\sum_i \hat G_i'\hat W_i \hat G_i\right)^{-1}\left(\frac{1}{n}\sum_i \hat G_i'\hat W_i\hat y_i\right)
$$
The only meaningful difference being that we solve the model once to compute $\hat p_i=p(\mathbf{X}_i,\hat \theta^k)$ and the form of $\hat D$ (note that our initial choice of $\hat D_i$ is equivalent to the one above if we set $\hat{\lambda}_g=0$ for all $g$). In order not to overload the notation, there is no iteration counter $k$ on $\hat p_i,\hat W_i,\hat D_i,\hat Z_i,\hat G_i,\hat y_i$, but of course these differ for each iteration. 

If the initial estimator $\hat \theta^0$ is sufficiently close to the true value $\hat \theta_{mle}$, then these iterations quickly converge to the maximum likelihood estimator.

